# Project 4: Decision Trees and Random Forest

### Name:
### Course Level:

**Introduction:**
* In this project, we explore the application of classification using: a) Decision Tree and b) Random Forest.

**Objectives:**
* The objective of this project is to develop software modules for classification of Titanic passengers via decision trees and random forests

## All Students

* The first problem we aim to us Information Gain to build a decision tree for classification.  The tree will complete when one of two conditions are met:

1. The maximum tree depth is reached.
2. The minimum number of samples is reached in a given leaf node.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import random
from collections import Counter

**Problem A (70pts)**

1 (5pts). The first thing you'll need to do is read the Titatinic dataset from D2L (details about the dataset can be found [Here](https://www.kaggle.com/c/titanic/data?select=test.csv)).  Note that you will need both the train.csv (to build your tree) and test.csv to evaluate the classificaiton accuracy.

In [2]:
# Read in the data #
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

2 (20pts). Next, we need a couple helper functions to compute the conditional entropy and information gain.

**Recall: Entropy** 
$$
    H(X) = -\sum_{\textbf{x} \in X} p(\textbf{x}) \log_2( p(\textbf{x}) )
$$

**Recall: Conditional Entropy** 
$$
    H(Y|X=\textbf{x}) = -\sum_{\textbf{y} \in Y} p(\textbf{y}|\textbf{x}) \log_2 ( p(\textbf{y}|\textbf{x}) )
$$
where
$$
    p(\textbf{y}|\textbf{x}) = \frac{p(\textbf{y},\textbf{x})}{p(\textbf{x})}
$$
and
$$
    p(\textbf{x}) = \sum_{\textbf{y} \in Y} p(\textbf{y},\textbf{x})
$$

**Recall: Information Gain** 
$$
    IG(Y|X) = H(Y) - H(Y|X)
$$



In [3]:
# Function definitions here (you may want to modify the input parameters depending on your implimentation #
def Entropy(d_X,d_Y):
    counts = Counter(d_Y) # Number of times each label appears
    total = len(d_Y)      # Number of samples
    H = -sum((count/total) * np.log2(count/total) for count in counts.values() if count > 0)
    return H

def C_Entropy(d_X,d_Y):
    total = len(d_Y)
    unique_vals = set(d_X) # Unique vals for the feature we're splitting on
    cH = 0.0
    for val in unique_vals:
        # For each unique value in the feature, find the corresponding labels in d_Y
        subset_y = [d_Y[i] for i in range(total) if d_X[i] == val]
        weight = len(subset_y) / total
        cH += weight * Entropy(d_X, subset_y) # Add weighted entropy of subset
    return cH

def IG(d_X,d_Y):
    IG = Entropy(d_X, d_Y) - C_Entropy(d_X, d_Y)
    return IG

3 (30pts). Define a function to build the tree, recall there should be one of two exit criterion:

1. The maximum tree depth is reached
2. The minimum number of samples is reached in a given leaf node.


In [4]:
# Structure of tree node
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, prediction=None, is_numeric=False):
        self.feature = feature              # Feature that node splits on
        self.threshold = threshold          # Threshold value (for numerical splits)
        self.left = left                    # Left child (<= threshold or == value)
        self.right = right                  # Right child (> threshold or != value)
        self.prediction = prediction        # Label prediction (for leaf nodes)
        self.is_numeric = is_numeric        # True if the split feature is numeric

# Returns the most common class label in a given list.
# Used when creating a leaf node to make a prediction.
def majority(l_labels):
    return Counter(l_labels).most_common(1)[0][0]

# Finds the best feature and split point that maximizes information gain.
def find_best_split(d_data, l_labels):
    best_feature = None
    best_threshold = None
    best_gain = -1
    is_numeric_split = False

    for feature in d_data.columns:
        X_col = d_data[feature].tolist()

        if isinstance(X_col[0], (int, float)):  # If feature is numeric
            unique_values = sorted(set(X_col))
            # Compute mdpts between all adj unique values
            thresholds = [(unique_values[i] + unique_values[i+1])/2 for i in range(len(unique_values)-1)]

            for threshold in thresholds:
                # Split labels based on threshold
                left_y = [l_labels[i] for i in range(len(X_col)) if X_col[i] <= threshold]
                right_y = [l_labels[i] for i in range(len(X_col)) if X_col[i] > threshold]

                # Screw invalid splits
                if not left_y or not right_y:
                    continue

                total = len(l_labels)
                # Weighted conditional entropy for split
                cH = (len(left_y)/total) * Entropy(None, left_y) + (len(right_y)/total) * Entropy(None, right_y)
                gain = Entropy(None, l_labels) - cH

                # Update best split if current is better
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
                    is_numeric_split = True
        else:  # If feature is categorical
            gain = IG(X_col, l_labels)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = X_col[0]  # Val to test equality against
                is_numeric_split = False

    return best_feature, best_threshold, best_gain, is_numeric_split

# Recursively build the decision tree
def BuildTree(d_data, l_labels, depth=0, max_depth=5, min_samples=10):
    # Stopping criteria: all labels are the same, or we've hit a depth/sample limit
    if len(set(l_labels)) == 1 or depth >= max_depth or len(d_data) <= min_samples:
        return Node(prediction=majority(l_labels))  # Make a leaf node with majority label

    # Find best split
    feature, threshold, gain, is_numeric = find_best_split(d_data, l_labels)

    # If no split improves gain, make a leaf node
    if gain == 0 or feature is None:
        return Node(prediction=majority(l_labels))

    # Split data on best feature and threshold
    if is_numeric:
        left_indices = d_data[feature] <= threshold
        right_indices = d_data[feature] > threshold
    else:
        left_indices = d_data[feature] == threshold
        right_indices = d_data[feature] != threshold

    # Slice dataset and labels accordingly
    left_data = d_data[left_indices]
    right_data = d_data[right_indices]
    left_labels = [l_labels[i] for i in range(len(l_labels)) if left_indices.iloc[i]]
    right_labels = [l_labels[i] for i in range(len(l_labels)) if right_indices.iloc[i]]

    # Recursively build left and right subtrees
    left_subtree = BuildTree(left_data, left_labels, depth + 1, max_depth, min_samples)
    right_subtree = BuildTree(right_data, right_labels, depth + 1, max_depth, min_samples)

    # Return a node with references to its subtrees
    return Node(feature=feature, threshold=threshold, left=left_subtree, right=right_subtree, is_numeric=is_numeric)

4 (15pts). Next write a function to perform the prediciton (classificaiton) of a new training sample:

In [5]:
# Predict the class label of a single sample using a trained decision tree.
def DT_predict(d_tree, d_sample):
    # Traverse until we reach a leaf node with a prediction
    while d_tree.prediction is None:
        # If current node splits on numeric feature
        if d_tree.is_numeric:
            # Go left if sample value is less <= threshold
            if d_sample[d_tree.feature] <= d_tree.threshold:
                d_tree = d_tree.left
            else:
                d_tree = d_tree.right
        else:
            # For categorical features, test for equality
            if d_sample[d_tree.feature] == d_tree.threshold:
                d_tree = d_tree.left
            else:
                d_tree = d_tree.right

    # When we reach a leaf node, return its prediction
    return d_tree.prediction

5 (5pts). Using the test set, evaluate the accuracy of your classification tree.

In [6]:
# Accuracy = (# of correct predictions) / (total samples)
def accuracy(t_test, t_tree):
    correct = 0  # Num of correct predictions
    
    for _, row in t_test.iterrows():
        true_label = row["Survived"]       # Actual label
        sample = row.drop("Survived")      # Feature values only
        pred = DT_predict(t_tree, sample)  # Predict using decision tree
        correct += (pred == true_label)    # Count+ if prediction is correct

    a_accuracy = correct / len(t_test)
    return a_accuracy

# Preprocesses the Titanic dataset & remove irrelevant features and handle missing data + encoding.
def preprocess(df):
    df = df.copy()

    # Drop columns not used in our model
    df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

    # Fill missing values in 'Age' with median age
    df["Age"] = df["Age"].fillna(df["Age"].median())

    # Fill missing values in 'Embarked' with the most frequent embarkation point
    df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

    # Convert categorical features into numeric
    df["Sex"] = df["Sex"].map({"male": 1, "female": 0})
    df["Embarked"] = df["Embarked"].map({"C": 0, "Q": 1, "S": 2})

    return df

# Apply preprocessing to train and test data
train_data = preprocess(train_df)
test_data = preprocess(test_df)

# Separate features and labels from training data
X_train = train_data.drop(columns=["Survived"])
y_train = train_data["Survived"].tolist()

# Build decision tree using train data
DTree = BuildTree(X_train, y_train)

# If 'Survived' is missing in test data, add dummy vals to enable evaluation
if "Survived" not in test_data.columns:
    test_data["Survived"] = [0] * len(test_data)  # dummy labels

# Evaluate model on the test data
a_accuracy = accuracy(test_data, DTree)
print(f"Accuracy: {a_accuracy * 100} %")

Accuracy: 77.03349282296651 %


**Problem B (30pts)**
* In Problem A, you designed a function to build a decision tree.  In this problem, you will use that function to build several different trees via bagging (bootstrapping and aggregation).  The result will be a random forest.  You should **at minimum** have a forest of at least 5 trees. 

* Once the forest is built, similar to A.5, design a function for comparing the accuracy of a standard tree, vs. random forest on the testing dataset.



In [7]:
# Generate a bootstrap sample (random sampling with replacement)
# Helps simulate the idea of training each tree on a slightly different dataset
def bootstrap_sample(df):
    indices = [random.randint(0, len(df) - 1) for _ in range(len(df))]
    return df.iloc[indices]

# Build Random Forest using bootstrapped datasets and random feature subsets
def BuildRandomForest(data, labels, num_trees, max_depth):
    min_samples = 5
    feature_subset_ratio = 0.6
    
    forest = []

    for _ in range(num_trees):
        # Bootstrap data: random rows sampled with replacement
        indices = np.random.choice(len(data), size=len(data), replace=True)
        sample_data = data.iloc[indices]
        sample_labels = [labels[i] for i in indices]

        # Random subspace: randomly select a subset of features for this tree
        features = data.columns.tolist()
        subset_size = max(1, int(len(features) * feature_subset_ratio))
        selected_features = np.random.choice(features, size=subset_size, replace=False)

        # Build decision tree using sampled data and selected features
        tree = BuildTree(sample_data[selected_features], sample_labels,
                         max_depth=max_depth, min_samples=min_samples)

        # Store tree along with features trained on
        forest.append((tree, selected_features))

    return forest

# Predict label of a sample using trees in forest
def RF_predict(forest, sample):
    predictions = []

    for tree, features in forest:
        # Extract only the subset of features this tree was trained on
        tree_sample = sample[features]
        predictions.append(DT_predict(tree, tree_sample))

    # Return the most common prediction (majority vote)
    return Counter(predictions).most_common(1)[0][0]

# Evaluate accuracy of random forest on test dataset
def RF_accuracy(test_data, forest):
    correct = 0
    total = len(test_data)

    for idx, row in test_data.iterrows():
        true_label = row["Survived"]        # Ground truth
        sample = row.drop("Survived")       # Feature values only
        pred = RF_predict(forest, sample)   # Predict using the forest

        if pred == true_label:
            correct += 1

    return correct / total

# Building and Evaluating
print("Wait! This may take a while :)")

# Train random forest of 5 trees using training data
forest = BuildRandomForest(X_train, y_train, num_trees=50, max_depth=10)

# Call accuracy from A.5 using single decision tree
dt_acc = accuracy(test_data, DTree)

# Call accuracy from B using random forest
rf_acc = RF_accuracy(test_data, forest)

# Print both results for comparison
print(f"Decision Tree Accuracy: {dt_acc * 100} %")
print(f"Random Forest Accuracy: {rf_acc * 100} %")

Wait! This may take a while :)
Decision Tree Accuracy: 77.03349282296651 %
Random Forest Accuracy: 77.27272727272727 %
